# Particle Set


Simulate trajectories of a particle cloud in a two-dimensional flow field.
A doubly-periodic domain and randomly-generated flow fields are initially used.
For additional documentation e.g. see :
[1](https://JuliaClimate.github.io/IndividualDisplacements.jl/dev/),
[2](https://JuliaClimate.github.io/MeshArrays.jl/dev/),
[3](https://docs.juliadiffeq.org/latest/solvers/ode_solve.html),
[4](https://en.wikipedia.org/wiki/Displacement_(vector))

Exercises:
- change the initial distribution of partices
- increase the duration of the trajectories simulation
- treat the non-periodic domain case by padding `u,v` with zeros
- replace `u,v` with your own two-dimensional flow fields

![particles in random flow](https://github.com/JuliaClimate/IndividualDisplacements.jl/raw/master/examples/figs/RandomFlow.gif)

## 1. Import Software

In [1]:
using IndividualDisplacements, DataFrames
p=dirname(pathof(IndividualDisplacements))
include(joinpath(p,"../examples/helper_functions.jl"))

Main.##256.isosurface

## 2. Flow Field

The `u,v` arrays below can be replaced with any other pair provided by the user.
A couple of important considerations, however:

- `u,v` are staggered on a C-grid; by `-0.5` grid point in direction `1` for `u` (`2` for `v`)
 from the grid cell center (0.5,0.5)
- `u,v` here derive from streamfunction `ϕ`, defined at the corner point, which ensures that
 the resulting `u,v` is non-divergent, purely rotational, over the C-grid domain.
In brief:

```
u=-(circshift(ϕ, (0,-1))-ϕ)
v=(circshift(ϕ, (-1,0))-ϕ)
```

In [2]:
u,v,ϕ=setup_random_flow()

([0.0011141240808857991 -0.003043653251059604 … 0.003715850347719228 0.001003795082372149; 0.0015100024550059737 0.0005317661059341811 … 0.003732179017135913 0.005872315653494289; … ; 0.0010025649163149485 -0.003121934042476751 … 0.003684426113682554 0.0008442958199324363; -0.0018988062691702688 -0.0019022782607809527 … 0.0003424436343205453 0.002269645528173884], [0.006131441256111789 0.005735562881991615 … 0.011016290496650614 0.01099996182723393; 0.009050041392848283 0.0039797340639426 … 0.0160875437244302 0.010462506070258643; … ; -0.0012047817194360752 0.001696589466049142 … -0.0031214144905566364 0.0002205679888053723; 0.0053082159191505315 0.0022952855690944636 … 0.00741577218674748 0.004042365473348797], [0.027304824849545672 0.026190700768659873 … 0.03202447027963705 0.02830861993191782; 0.03343626610565746 0.03192626365065149 … 0.04304076077628766 0.03930858175915175; … ; 0.023201390649831215 0.022198825733516267 … 0.027730112583446206 0.02404568646976365; 0.02199660893039514

If user were to start with collocated velocity (`uC,vC` at the grid cell center) then
one can easily obtain the staggered velocity (`u,v`) as follows. These may contain both
[rotational and divergent](https://en.wikipedia.org/wiki/Helmholtz_decomposition) components.

```
u=0.5*(circshift(uC, (0,1))+uC)
v=0.5*(circshift(vC, (1,0))+vC)
```

## 3. Initialize Individuals

For example, we can initialize 100 particles within a central subdomain as follows.

In [3]:
np,nq=size(u)
x=np*(0.4 .+ 0.2*rand(100))
y=nq*(0.4 .+ 0.2*rand(100));

The `setup_point_cloud` function then wraps everything in the `Individuals` data structure.

In [4]:
𝐼=setup_point_cloud(u,v,X=x,Y=y)
#𝐼.𝑃.𝑇[2]=1000.
𝐼.🔴

,ID,x,y,t
,Int64,Float64,Float64,Float64


## 3. Compute Trajectories

The time period is `𝐼.𝑃.𝑇` by default, unless `∫!(𝐼,𝑇)` is called instead.

In [5]:
∫!(𝐼)

1×100 Array{Array{Float64,1},2}:
 [6.92084, 10.1509, 1.0]  [6.57585, 7.79439, 1.0]  …  [7.10888, 7.91948, 1.0]

## 4. Plot Results

For example, generate a simple animation:

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*